In [9]:
import torch
from sklearn.metrics import DistanceMetric
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from scipy.spatial import distance

import cv2

import sys
sys.path.append('/workspaces/dbm25/task_1_2')
from tabulate import tabulate


from extract_features import extract_features



In [10]:
# Sample image path
sample_image_path = '/workspaces/dbm25/data/Part2/Part2/brain_glioma/brain_glioma_1003.jpg'
# Load gray scale image with cv2
image = cv2.imread(sample_image_path, cv2.IMREAD_GRAYSCALE)
rgb   = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

In [11]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
model.eval()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [12]:
from transformers import ViTConfig, ViTModel, ViTImageProcessor

# 1. Load the ViT config from your local folder
vision_config = ViTConfig.from_pretrained(
    "/workspaces/dbm25/data/vit_b16_224-mluke/vision_encoder"
)

# 2. Load the model weights into a ViTModel
vision_model = ViTModel.from_pretrained(
    "/workspaces/dbm25/data/vit_b16_224-mluke/vision_encoder",
    config=vision_config,
    # map_location="cpu",  # uncomment if you need CPU-only
)

# 3. Set to eval mode if you’re doing inference
vision_model.eval()

# 4. Prepare your images for ViT
processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")
# If you have PIL images in a list called `images`:
inputs = processor(images=rgb, return_tensors="pt")

outputs   = vision_model(**inputs)
# Print the keys of the output and their shapes
for key, value in outputs.items():
    print(f"{key}: {value.shape}")

2025-05-19 14:17:00.684110: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-19 14:17:00.685406: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-19 14:17:00.689776: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-19 14:17:00.699436: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747664220.715436     613 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747664220.72

last_hidden_state: torch.Size([1, 197, 768])
pooler_output: torch.Size([1, 768])


In [13]:
# Load feature dictionary from /workspaces/dbm25/data/extracted_features.pt
feature_list = torch.load('/workspaces/dbm25/data/extracted_features.pt')

In [14]:
print(feature_list[0].keys())

dict_keys(['file_path', 'class', 'cm', 'hog', 'avgpool', 'layer3', 'fc'])


In [15]:
print(feature_list[0])

{'file_path': '/workspaces/dbm25/data/Part1/Part1/brain_glioma/brain_glioma_0051.jpg', 'class': 'brain_glioma', 'cm': tensor([ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  3.3333e-03,
         3.3222e-03,  1.7234e+01,  4.6667e-02,  7.7822e-02,  7.1705e+00,
         2.3333e-02,  3.6122e-02,  8.8549e+00,  5.3333e-02,  7.7156e-02,
         5.6383e+00,  5.0000e-02,  6.7500e-02,  5.6880e+00,  4.3333e-02,
         4.1456e-02,  4.4858e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         1.8467e+00,  7.2098e+00,  3.2567e+00,  2.6333e+00,  3.5589e+00,
         3.3027e-01,  1.7380e+01,  1.1373e+03,  2.5840e+00,  4.6560e+01,
         2.3561e+03,  1.0891e+00,  6.5000e+01,  2.2524e+03,  6.9589e-01,
         6.6523e+01,  1.8115e+03,  8.6714e-01,  5.6347e+01,  3.1201e+03,
         9.5670e-01,  1.5770e+01,  1.5331e+03,  2.9974e+00,  1.9967e+00,
         2.1567e+00,  4.5398e+00,  6.0667e-01,  3.6529e-01,  4.4642e-01,
      

In [16]:
print(feature_list[0]['fc'].shape)

torch.Size([1000])


In [17]:
fc_list = []
for i in range(len(feature_list)):
    fc_list.append(feature_list[i]['fc'].numpy())


print(len(fc_list))
print(fc_list[0].shape)

X = np.stack([f for f in fc_list], axis=0)

print(X.shape)

3006
(1000,)
(3006, 1000)


In [18]:
# given that feature list is a list of dicts with dict_keys(['file_path', 'class', 'cm', 'hog', 'avgpool', 'layer3', 'fc'])
# Convert it into a pandas dataframe
df = pd.DataFrame(feature_list)

In [19]:
# Show the first 5 rows of the dataframe
df.head()

,file_path,class,cm,hog,avgpool,layer3,fc
0,/workspaces/dbm25/data/Part1/Part1/brain_gliom...,brain_glioma,"[tensor(0.), tensor(0.), tensor(0.), tensor(0....","[tensor(0., dtype=torch.float64), tensor(0., d...","[tensor(0.8356), tensor(0.1613), tensor(0.5870...","[tensor(0.1084), tensor(0.0177), tensor(0.0076...","[tensor(-0.1851), tensor(2.3174), tensor(-0.96..."
1,/workspaces/dbm25/data/Part1/Part1/brain_gliom...,brain_glioma,"[tensor(0.), tensor(0.), tensor(0.), tensor(0....","[tensor(0., dtype=torch.float64), tensor(0., d...","[tensor(0.4021), tensor(0.1057), tensor(0.3497...","[tensor(0.1296), tensor(0.0262), tensor(0.0140...","[tensor(-0.4316), tensor(0.8583), tensor(-1.41..."
2,/workspaces/dbm25/data/Part1/Part1/brain_gliom...,brain_glioma,"[tensor(0.), tensor(0.), tensor(0.), tensor(0....","[tensor(0., dtype=torch.float64), tensor(0., d...","[tensor(0.5315), tensor(0.0803), tensor(0.2933...","[tensor(0.1146), tensor(0.0129), tensor(0.0106...","[tensor(-1.0882), tensor(1.2385), tensor(-1.31..."
3,/workspaces/dbm25/data/Part1/Part1/brain_gliom...,brain_glioma,"[tensor(0.), tensor(0.), tensor(0.), tensor(1....","[tensor(0., dtype=torch.float64), tensor(0., d...","[tensor(0.3852), tensor(0.2432), tensor(0.4104...","[tensor(0.1011), tensor(0.0200), tensor(0.0136...","[tensor(-1.1115), tensor(0.7664), tensor(-1.43..."
4,/workspaces/dbm25/data/Part1/Part1/brain_gliom...,brain_glioma,"[tensor(0.), tensor(0.), tensor(0.), tensor(0....","[tensor(0., dtype=torch.float64), tensor(0., d...","[tensor(0.5562), tensor(0.0179), tensor(0.5070...","[tensor(0.1269), tensor(0.0230), tensor(0.0161...","[tensor(-0.2769), tensor(0.6544), tensor(-1.03..."


In [20]:
# Show unique possible classes
unique_classes = df['class'].unique()
print(unique_classes)

['brain_glioma' 'brain_tumor' 'brain_menin']


In [21]:
# unique classes are ['brain_glioma' 'brain_tumor' 'brain_menin'] map them to 0, 1, 2
class_mapping = {
    'brain_glioma': 0,
    'brain_tumor': 1,
    'brain_menin': 2
}
# Map the dataframe class column to the new mapping
df['class'] = df['class'].map(class_mapping)
# Show the first 5 rows of the dataframe
df.head()

,file_path,class,cm,hog,avgpool,layer3,fc
0,/workspaces/dbm25/data/Part1/Part1/brain_gliom...,0,"[tensor(0.), tensor(0.), tensor(0.), tensor(0....","[tensor(0., dtype=torch.float64), tensor(0., d...","[tensor(0.8356), tensor(0.1613), tensor(0.5870...","[tensor(0.1084), tensor(0.0177), tensor(0.0076...","[tensor(-0.1851), tensor(2.3174), tensor(-0.96..."
1,/workspaces/dbm25/data/Part1/Part1/brain_gliom...,0,"[tensor(0.), tensor(0.), tensor(0.), tensor(0....","[tensor(0., dtype=torch.float64), tensor(0., d...","[tensor(0.4021), tensor(0.1057), tensor(0.3497...","[tensor(0.1296), tensor(0.0262), tensor(0.0140...","[tensor(-0.4316), tensor(0.8583), tensor(-1.41..."
2,/workspaces/dbm25/data/Part1/Part1/brain_gliom...,0,"[tensor(0.), tensor(0.), tensor(0.), tensor(0....","[tensor(0., dtype=torch.float64), tensor(0., d...","[tensor(0.5315), tensor(0.0803), tensor(0.2933...","[tensor(0.1146), tensor(0.0129), tensor(0.0106...","[tensor(-1.0882), tensor(1.2385), tensor(-1.31..."
3,/workspaces/dbm25/data/Part1/Part1/brain_gliom...,0,"[tensor(0.), tensor(0.), tensor(0.), tensor(1....","[tensor(0., dtype=torch.float64), tensor(0., d...","[tensor(0.3852), tensor(0.2432), tensor(0.4104...","[tensor(0.1011), tensor(0.0200), tensor(0.0136...","[tensor(-1.1115), tensor(0.7664), tensor(-1.43..."
4,/workspaces/dbm25/data/Part1/Part1/brain_gliom...,0,"[tensor(0.), tensor(0.), tensor(0.), tensor(0....","[tensor(0., dtype=torch.float64), tensor(0., d...","[tensor(0.5562), tensor(0.0179), tensor(0.5070...","[tensor(0.1269), tensor(0.0230), tensor(0.0161...","[tensor(-0.2769), tensor(0.6544), tensor(-1.03..."


In [22]:
# ── 1) Suppose df is your DataFrame
# It has 3006 rows and df['fc'] is a torch.Tensor of shape (1000,)
# ────────────────────────────────────────────────────────────────────────

# Convert the column of tensors into an (N, 1000) NumPy array
# (make sure each tensor is on CPU first)
fc_arrays = np.stack([
    t.cpu().numpy()
    for t in df['layer3'].tolist()
])  # shape: (3006, 1000)

# ── 2) Run KMeans
# Pick the number of clusters you want, e.g. k = 5
k = 3
km = KMeans(n_clusters=k, random_state=0)
km.fit(fc_arrays)

# km.cluster_centers_  → shape (k, 1000)
# km.labels_           → array of length 3006
centroids = km.cluster_centers_
labels = km.labels_

# ── 3) Add the labels back to your DataFrame
df['cluster'] = km.labels_

In [23]:
# Check how many of each class are in each cluster
cluster_counts = df.groupby(['class', 'cluster']).size().unstack(fill_value=0)
print(cluster_counts)

cluster    0    1    2
class                 
0         11  460  531
1        851   11  140
2        218  216  568


In [25]:
# Sample image path
sample_image_path = '/workspaces/dbm25/data/Part2/Part2/brain_glioma/brain_glioma_1009.jpg'

test_features = extract_features(sample_image_path, model)
# Check in which cluster the test image is
test_features_layer3 = test_features['layer3'].cpu().numpy()
# fit the test features to the kmeans model
test_label = km.predict(test_features_layer3.reshape(1, -1))
print(test_label)


Processing /workspaces/dbm25/data/Part2/Part2/brain_glioma/brain_glioma_1009.jpg
[2]


In [ ]:
# Check in which cluster the test image is
test_features_layer3 = test_features['layer3'].cpu().numpy()
# fit the test features to the kmeans model
test_label = km.predict(test_features_layer3.reshape(1, -1))
print(test_label)


[2]


In [ ]:
fc_class_list = []
for i in range(len(feature_list)):
    fc_class_list.append(feature_list[i]['class'].numpy())
    fc_class_list.append(feature_list[i]['fc'].numpy())

In [ ]:
k = 3
kmeans = KMeans(n_clusters=k, random_state=0).fit(X)

centroids = kmeans.cluster_centers_
labels = kmeans.labels_


(3, 1000)
(3006,)
[1 1 1 ... 2 1 1]


In [ ]:
### TODO Add scikit learn rand_score 